In [2]:
import json
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.tokenize import word_tokenize
import spacy
import os
import re 
import sklearn 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stpwds = set(stopwords.words('english'))
#nltk.download('wordnet')
nlp = spacy.load('C://Users//seanp//Anaconda3//Lib//site-packages//en_core_web_sm//en_core_web_sm-2.0.0')
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from nltk.stem.porter import *
porter_stemmer = PorterStemmer()
import nltk.data
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
from nltk.tokenize import TreebankWordTokenizer
treebank_tokenizer = TreebankWordTokenizer()
from nltk.util import bigrams 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\seanp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
wd = os.getcwd() 
# grab the locations of those lil jsons
json_files = [pos_json for pos_json in os.listdir(wd + '\\diag_jsons\\') if pos_json.endswith('.json')]

In [7]:
with open(wd+ '\\diag_jsons\\'+json_files[0], "r") as read_file:
    data = json.load(read_file)
json_files[0].replace('.json','')

'10-Things-I-Hate-About-You_script'

In [4]:
def remove_multiple_strings(cur_string, replace_list,low= False):
    for cur_word in replace_list:
        if low == False:
            cur_string = cur_string.replace(cur_word, '')
        else:
            cur_string = cur_string.replace(cur_word.lower(), '')
    return cur_string

In [5]:
# Original Process with LEMMATIZATION but no bigrams (we do remove all character names, we will also want to get rid of stopwords )
slim_pos = ['JJ','JJR','JJS','NN','NNS','RB','RBR','RBS','VB','VBD','VBG','VBN','VBP','VBZ']
fat_pos = ['JJ','JJR','JJS','NN','NNS','NNP','NNPS','RB','RBR','RBS','VB','VBD','VBG','VBN','VBP','VBZ','WDT','WP','WRB']

mov_dict = {}
countar = 0
char_series = pd.Series()
for fil in json_files:
    countar +=1
    if countar == 10:
        print('We have processed 10 scripts')
    
    # read the json file from the directory 
    with open(wd+ '\\diag_jsons\\'+fil, "r") as read_file:
        data = json.load(read_file)
        
    mov_dict[fil.replace('.json','')] = data
    # now that we have done this 
    # we may as well start loading up our pandas series with the character texts.
    # so we will start iterating through each character in the new json file 
    # added to our dictionary and make each character a new iterm in our series 
    # which will be indexed by the title of the script/movie and the character's name 
    # with four '_'s between them 
    # so we can do easy retrieval of which character was from which movie :)
    mov_chars = mov_dict[fil.replace('.json','')]['dialogues'].keys()
    for char in mov_dict[fil.replace('.json','')]['dialogues'].keys():
        #char_series.at[char+'____'+fil.replace('.json','')] = mov_dict[fil.replace('.json','')]['dialogues'][char]
        char_series.at[char+'____'+fil.replace('.json','')] = re.sub("\\n\[\d+\]",'',mov_dict[fil.replace('.json','')]['dialogues'][char])
        # until I think of a better way of replacing the names, 
        # I have to make the entire text lowercase 
        # so that way I get all instances of the character names being used.... 
        char_series.at[char+'____'+fil.replace('.json','')] = char_series.at[char+'____'+fil.replace('.json','')].lower()
        # now we want to remove the names of the characters, because those seemed to be too important in creating clusters
        char_series.at[char+'____'+fil.replace('.json','')] = remove_multiple_strings(char_series.at[char+'____'+fil.replace('.json','')],mov_chars,low=True)
        
        
        tok_chars = sentence_tokenizer.tokenize(char_series.at[char+'____'+fil.replace('.json','')])
            # so... we need to do this for every sentence.... YAY.... 
        lemz = ''
        # I think I need to get rid of punctuation unless lemmatization gets rid of it 
        for sentence in tok_chars:
            toks = treebank_tokenizer.tokenize(sentence)
            narrow_toks = [word[0] for word in nltk.pos_tag(toks) if word[1] in slim_pos]
            #lemmatized_words = [wordnet_lemmatizer.lemmatize(token) for token in toks]
            lemz += " ".join(narrow_toks) + ' '
            lemz = [w for w in treebank_tokenizer.tokenize(lemz) if w not  in stpwds]
            lemz = " ".join(lemz) + ' '

            #lemz = [tok for tok in nlp(lemz) if tok.pos_ in ['NOUN','VERB','ADV','ADJ'] ]
            
        char_series.at[char+'____'+fil.replace('.json','')] = lemz
        
        # we get rid of those pesky \n[number] patterns that denote a line change 
        # which I suppose I can add back IN as a second column if necessary , but that's just going to be unnecessary for the clustering!

# I'm guessing that every thime there is a new line it will go with that kind of scheme 
# so I suppose my job will be to remove those bois if they still occur after tokenization.... hoping they just dissapear, lmao !!!

# also, very nice, I can iterate over the ['dialogues.keys()']
# this is VERY fun!!!! 

We have processed 10 scripts


In [26]:
# Define helper functions, These are from Stephen
def get_keys(topic_matrix):
    '''returns an integer list of predicted topic categories for a given topic matrix'''
    keys = []
    for i in range(topic_matrix.shape[0]):
        keys.append(topic_matrix[i].argmax())
    return keys
###################################################################################
def keys_to_counts(keys):
    '''returns a tuple of topic categories and their accompanying magnitudes for a given list of keys'''
    count_pairs = Counter(keys).items()
    categories = [pair[0] for pair in count_pairs]
    counts = [pair[1] for pair in count_pairs]
    return (categories, counts)

##############################################################################
def get_top_n_words(n, keys, document_term_matrix, count_vectorizer):
    '''returns a list of n_topic strings, where each string contains the n most common 
        words in a predicted category, in order'''
    top_word_indices = []
    for topic in range(n_topics):
        temp_vector_sum = 0
        for i in range(len(keys)):
            if keys[i] == topic:
                temp_vector_sum += document_term_matrix[i]
        temp_vector_sum = temp_vector_sum.toarray()
        top_n_word_indices = np.flip(np.argsort(temp_vector_sum)[0][-n:],0)
        top_word_indices.append(top_n_word_indices)   
    top_words = []
    for topic in top_word_indices:
        topic_words = []
        for index in topic:
            temp_word_vector = np.zeros((1,document_term_matrix.shape[1]))
            temp_word_vector[:,index] = 1
            the_word = count_vectorizer.inverse_transform(temp_word_vector)[0][0]
            topic_words.append(the_word.encode('ascii').decode('utf-8'))
        top_words.append(" ".join(topic_words))         
    return top_words

In [31]:
count_vectorizer = CountVectorizer(stop_words='english', max_features=10000)
text_sample = char_series.as_matrix()
# so we essenatially create our document_term matrix.... 
#display(small_text_sample)
document_term_matrix = count_vectorizer.fit_transform(text_sample)
display(document_term_matrix)

n_topics = 40

lsa_model = TruncatedSVD(n_components=n_topics)
lsa_topic_matrix = lsa_model.fit_transform(document_term_matrix)
   # display(lsa_topic_matrix)

lsa_keys = get_keys(lsa_topic_matrix)
lsa_categories, lsa_counts = keys_to_counts(lsa_keys)

top_n_words_lsa = get_top_n_words(20, lsa_keys, document_term_matrix, count_vectorizer)

for i in range(len(top_n_words_lsa)):
    print("Topic {}: ".format(i), top_n_words_lsa[i])


C:\Users\seanp\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


<11075x10000 sparse matrix of type '<class 'numpy.int64'>'
	with 1341714 stored elements in Compressed Sparse Row format>

Topic 0:  know got want right ve think going good come time let look tell man say yeah okay way need gon
Topic 1:  got fuck gon fuckin shit yeah ta ai let man hey guy goin barry oh double fucking ass swamp die
Topic 2:  door looks man turns hand room eyes face got head away open comes car pulls takes starts looking gun look
Topic 3:  looks turns yeah hey door takes eyes walks stares car sees hand away camera floor pulls steps starts shower curtain
Topic 4:  07 revision pink 20 white 19 yellow seconds war run coffee bridge sheets african stocks nation american oldest la event
Topic 5:  schultz dr know nigger horse man good big little got looks ai german candie white dollars says mr yes say
Topic 6:  fuck fucking know shit fuckin man bitch little gon shut god look yeah tell thing hell want car right holy
Topic 7:  man hey fucking dr yeah authorization really fuck stones ass shit wow little good high sorry god wrong dad hell
Topic 8:  dr going jones schultz box right hot got die quite wan

In [ ]:
#''.join([i for i in X if not i.isdigit()])
# THIS IS IT BOYS!!!! HOW THE NUMBERS GO AWAY!!! 
char_series = pd.DataFrame(char_series)
for index, row in char_series.iterrows():
    char_series.loc[index,row[0]] = ''.join([i for i in row[0] if not i.isdigit()])

In [1]:
char_series

NameError: name 'char_series' is not defined